In [142]:
import peewee as pw
import pandas as pd
import numpy as np
import structure

In [2]:
def df_from_comsol(name):
    with open(structure.path+ r'\Batch\io\Data.csv', 'r') as file:
        lines = file.readlines()
    lines = lines[4:]

    lines[:] = (line.replace('\n', '') for line in lines[:])
    lines[0] = lines[0].replace('% ', '')

    # pd.DataFrame( lines[1:],columns=lines[0])
    data = [line.split(',') for line in lines]
    return pd.DataFrame(data=data[1:], columns=data[0]).astype(float)
    

In [3]:
df = df_from_comsol('test')
df.head()

,Ke,KH,Kr,Kdisp,KqH,Ks,Kd,Kc,Kp,KrD,...,Q,DH,QHD,QHH,prod,Qm,DHp,QH,D,QD
0,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0
1,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,2.000000e-12,0.0,0.0,0.0,0.0,0.0
2,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,4.000000e-12,0.0,0.0,0.0,0.0,0.0
3,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,8.000000e-12,0.0,0.0,0.0,0.0,0.0
4,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,1.600000e-11,0.0,0.0,0.0,0.0,0.0


In [245]:
def distribute(df: pd.DataFrame, name='test', desc='test desc'):
    consts = [i for i in df.columns if 'K' in i]
    nonconsts = [i for i in df.columns if 'K' not in i]
    combs = df.drop_duplicates(subset=consts)[consts]
    combs = combs.reset_index(drop=True)
    notes = []
    for num, comb in combs.iterrows():

        df_filtered = df
        for key, value in comb.items():
            df_filtered = df_filtered[df_filtered[key] == value]

        const_vals = dict(zip(comb.keys(), comb.values))
        notes.append((const_vals, df_filtered[nonconsts]))
    return notes

In [247]:
a= distribute(df)